---

Created for [Pricing and Hedging Derivative Securities: Theory and Methods](https://book.derivative-securities.org/)

Authored by
- Kerry Back, Rice University
- Hong Liu, Washington University in St. Louis
- Mark Loewenstein, University of Maryland
 
---

<a target="_blank" href="https://colab.research.google.com/github/math-finance-book/book-code/blob/main/17_StochasticVol.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<img src="https://www.dropbox.com/scl/fi/6hwvdff7ajaafmkpmnp0o/under_construction.jpg?rlkey=3dex2dx86anniqoutwyqashnu&dl=1" alt="Under Construction" width="400"/>


In [ ]:
import numpy as np
import pandas as pd

def simulating_stochastic_volatility(S, V0, r, q, dt, N, theta, kappa, sigma, rho):
    """
    Inputs:
    S = initial stock price
    V0 = initial variance
    r = risk-free rate
    q = dividend yield
    dt = length of each time period (Delta t)
    N = number of time periods
    theta = long-term variance (mean of variance)
    kappa = rate of mean reversion of variance
    sigma = volatility of variance
    rho = correlation between the two Wiener processes
    """
    LogS = np.log(S)
    Sqrdt = np.sqrt(dt)
    
    time = np.zeros(N + 1)
    stock_price = np.zeros(N + 1)
    volatility = np.zeros(N + 1)
    
    stock_price[0] = S
    volatility[0] = V0    
    
    for i in range(1, N + 1):
        time[i] = i * dt
        Z1 = np.random.randn()
        Z2 = np.random.randn()
        W1 = Z1
        W2 = rho * Z1 + np.sqrt(1 - rho**2) * Z2
        
        LogS = LogS + (r - q - 0.5 * volatility[i-1]**2) * dt + np.sqrt(volatility[i-1]**2 * dt) * W1
        S = np.exp(LogS)
        stock_price[i] = S
        
        volatility[i] = np.sqrt(np.maximum(volatility[i-1]**2 + kappa * (theta - volatility[i-1]**2) * dt + sigma * np.sqrt(volatility[i-1]**2 * dt) * W2, 0))

    df_stochastic_vol = pd.DataFrame({'Time': time, 'Stock Price': stock_price, 'Volatility': volatility})
    df_stochastic_vol.to_csv('stochastic_volatility_simulation.csv', index=False)
    return df_stochastic_vol

# Example usage:
S = 100       # Initial stock price
V0 = 0.04     # Initial variance
r = 0.05      # Risk-free rate
q = 0.02      # Dividend yield
dt = 1/252    # Length of each time period (daily)
N = 252       # Number of time periods (one year)
theta = 0.04  # Long-term variance
kappa = 2.0   # Rate of mean reversion of variance
sigma = 0.3   # Volatility of variance
rho = -0.7    # Correlation between the two Wiener processes

df_stochastic_vol = simulating_stochastic_volatility(S, V0, r, q, dt, N, theta, kappa, sigma, rho)
print(df_stochastic_vol)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Heston model parameters
S0 = 100     # Initial stock price
V0 = 0.04    # Initial variance
rho = -0.7   # Correlation between the two Wiener processes
kappa = 2.0  # Rate of mean reversion of variance
theta = 0.04 # Long-term variance
sigma = 0.3  # Volatility of variance
r = 0.05     # Risk-free rate
T = 1.0      # Time to maturity
N = 252      # Number of time steps
dt = T / N   # Time step size
n_simulations = 1000  # Number of simulations

# Preallocate arrays
S = np.zeros((N+1, n_simulations))
V = np.zeros((N+1, n_simulations))
S[0] = S0
V[0] = V0

# Simulate the paths
for t in range(1, N+1):
    Z1 = np.random.normal(size=n_simulations)
    Z2 = np.random.normal(size=n_simulations)
    W1 = Z1
    W2 = rho * Z1 + np.sqrt(1 - rho**2) * Z2
    
    V[t] = np.maximum(V[t-1] + kappa * (theta - V[t-1]) * dt + sigma * np.sqrt(V[t-1] * dt) * W2, 0)
    S[t] = S[t-1] * np.exp((r - 0.5 * V[t-1]) * dt + np.sqrt(V[t-1] * dt) * W1)

# Plot the results
plt.figure(figsize=(12, 6))
for i in range(n_simulations):
    plt.plot(S[:, i], lw=0.5, alpha=0.3)
plt.title('Stock Price Paths with Stochastic Volatility (Heston Model)')
plt.xlabel('Time Steps')
plt.ylabel('Stock Price')
plt.grid(True)
plt.show()

# Plot the volatility paths
plt.figure(figsize=(12, 6))
for i in range(n_simulations):
    plt.plot(V[:, i], lw=0.5, alpha=0.3)
plt.title('Variance Paths with Stochastic Volatility (Heston Model)')
plt.xlabel('Time Steps')
plt.ylabel('Variance')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, kurtosis

def simulating_stochastic_volatility(S, V0, r, q, dt, N, theta, kappa, sigma, rho, n_simulations):
    """
    Inputs:
    S = initial stock price
    V0 = initial variance
    r = risk-free rate
    q = dividend yield
    dt = length of each time period (Delta t)
    N = number of time periods
    theta = long-term variance (mean of variance)
    kappa = rate of mean reversion of variance
    sigma = volatility of variance
    rho = correlation between the two Wiener processes
    n_simulations = number of simulations
    """
    LogS = np.log(S)
    Sqrdt = np.sqrt(dt)
    
    log_returns = []
    
    for _ in range(n_simulations):
        stock_price = S
        variance = V0
        for _ in range(N):
            Z1 = np.random.randn()
            Z2 = np.random.randn()
            W1 = Z1
            W2 = rho * Z1 + np.sqrt(1 - rho**2) * Z2
            
            log_return = (r - q - 0.5 * variance) * dt + np.sqrt(variance * dt) * W1
            LogS = np.log(stock_price) + log_return
            stock_price = np.exp(LogS)
            
            variance = np.maximum(variance + kappa * (theta - variance) * dt + sigma * np.sqrt(variance * dt) * W2, 0)
            
            log_returns.append(log_return)
    
    return log_returns

# Example usage:
S = 100       # Initial stock price
V0 = 0.04     # Initial variance
r = 0.05      # Risk-free rate
q = 0.02      # Dividend yield
dt = 1/252    # Length of each time period (daily)
N = 252       # Number of time periods (one year)
theta = 0.04  # Long-term variance
kappa = 0.2   # Rate of mean reversion of variance
sigma = 0.3   # Volatility of variance
rho = -0.7    # Correlation between the two Wiener processes
n_simulations = 1000  # Number of simulations

log_returns = simulating_stochastic_volatility(S, V0, r, q, dt, N, theta, kappa, sigma, rho, n_simulations)

# Plotting the distribution of log returns
sns.histplot(log_returns, bins=100, kde=True, stat="density", color="blue", label="Simulated Log Returns")
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, np.mean(log_returns), np.std(log_returns))
plt.plot(x, p, 'k', linewidth=2, label="Normal Distribution")
plt.title('Distribution of Log Returns with Stochastic Volatility')
plt.xlabel('Log Return')
plt.ylabel('Density')
plt.legend()

# Display kurtosis
kurt = kurtosis(log_returns)
plt.figtext(0.15, 0.8, f'Kurtosis: {kurt:.2f}', fontsize=12)

plt.show()

In [ ]:


import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.stats import norm
import yfinance as yf

def indicator_function(condition):
    return 1 if condition else 0

def calculate_variance_swap_strike(S, r, T, options_data):
    # Extract call and put prices
    call_data = options_data[options_data['Type'] == 'call'].copy()
    put_data = options_data[options_data['Type'] == 'put'].copy()
    
    # Interpolate call and put prices
    strikes = np.unique(options_data['Strike'])
    call_interp = interp1d(call_data['Strike'], call_data['Price'], fill_value="extrapolate")
    put_interp = interp1d(put_data['Strike'], put_data['Price'], fill_value="extrapolate")
    
    # Integrate using numerical methods (trapezoidal rule)
    
    K_min, K_max = strikes.min(), strikes.max()
    K = np.linspace(K_min, S*np.exp(r*T), 500)
    integral1 = np.trapezoid(put_interp(K) / K**2, K)
    K = np.linspace(S*np.exp(r*T),K_max, 500)
    integral2 = np.trapezoid(call_interp(K) / K**2, K)
    
    # Calculate the variance swap strike
    variance_swap_strike = np.sqrt(2 * (integral1+integral2) / T)
    return variance_swap_strike

# Example usage
ticker = "AAPL"
S = 150.0  # Current stock price
r = 0.01  # Risk-free rate
T = 0.5  # Time to maturity (6 months)

# Fetch options data from Yahoo Finance
stock = yf.Ticker(ticker)
expiry = stock.options[0]
opt_chain = stock.option_chain(expiry)
calls = opt_chain.calls[['strike', 'lastPrice']].copy()
puts = opt_chain.puts[['strike', 'lastPrice']].copy()
calls.columns = ['Strike', 'Price']
puts.columns = ['Strike', 'Price']
calls['Type'] = 'call'
puts['Type'] = 'put'
options_data = pd.concat([calls, puts])

# Calculate the variance swap strike
variance_swap_strike = calculate_variance_swap_strike(S, r, T, options_data)
print(f"Variance Swap Strike: {variance_swap_strike:.4f}")